In [ ]:
import pandas as pd
import numpy as np
import altair as alt

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pd.set_option('max_colwidth', 400)
twitter_exemplos = pd.read_csv('https://gist.githubusercontent.com/davidedup/b2545e4d10112094730eaa73a6cfda92/raw/1882e4e11ca37c166d65036bc5dc964873778e93/twitter_dados.csv')
twitter_exemplos = twitter_exemplos.drop("user", axis=1)
twitter_exemplos[0:10]

,category,text
0,0,"That's a question for the Gov, but as you can see FL wil…"
1,1,"Update: Collision; highway WB: at Exit194, no lns clsd, Ocrd: // :19AM.| :36P"
2,0,SNAP E&amp;T Learning Academy builds national leadership for the program
3,0,What Do You Mean? Taking it worldwide . #14Days
4,1,Disabled Vehicle on Northbound highway NB at Mile Marker .
5,0,Direct hit! See how our #THAAD system intercepted a missile target in major U.S. test:
6,0,Shout-out to the Class of ' arriving in the burg today! General move-in is just days away. Who's…
7,1,NB highway after 84th St\nLeft Lane Blocked\nDue to a Crash\nKent County\n\nRandy Weits\n//\n:
8,1,Snoqualmie: Construction season is underway &amp; there are work zones on highway. Make sure &amp; check Travel Alerts page to
9,1,Moving work crew - NB highway from mile to Mile. Right lane closed.


In [ ]:
y = twitter_exemplos["category"].values
y

array([0, 1, 0, ..., 0, 1, 1])

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

# removendo caracteres especiais e salvando na variavel
clean_tweets = []
for w in range(len(twitter_exemplos.text)):
    lyric = twitter_exemplos['text'][w]

    lyric  = re.sub("(\\d|\\W)+|\w*\d\w*"," ", lyric)
    lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    clean_tweets.append(lyric)

clean_tweets[0:15]

['That question for the Gov but you can see wil',
 'Update Collision highway lns clsd Ocrd',
 'SNAP amp Learning Academy builds national leadership for the program',
 'What You Mean Taking worldwide Days',
 'Disabled Vehicle Northbound highway Mile Marker',
 'Direct hit See how our THAAD system intercepted missile target major test',
 'Shout out the Class arriving the burg today General move just days away Who',
 'highway after Left Lane Blocked Due Crash Kent County Randy Weits',
 'Snoqualmie Construction season underway amp there are work zones highway Make sure amp check Travel Alerts page',
 'Moving work crew highway from mile Mile Right lane closed',
 'Waiting for miracle food story survival from',
 'Incident Exit',
 'Thanks for the follow Keith Please report this issue here Thanks',
 'Update Construction Line Queens bound from Queensboro Plaza Station Willets Point Shea Stadium Station',
 'The perfect fit getting kitted out her official TeamUSA closing ceremony uniform']

In [ ]:
#TF-IDF vectorizer
tfv = TfidfVectorizer(
        min_df = 10,
        max_df = 0.5,
        max_features = None,
        stop_words = stop_words, 
        ngram_range = (1,3)
  )

#transform
vec_text = tfv.fit_transform(clean_tweets)

#returns a list of words.
words = tfv.get_feature_names()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vec_text, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=2, init_size=1024, batch_size=2048, random_state=20)
kmeans.fit(vec_text)
labels = kmeans.predict(vec_text)
tweets_ag = twitter_exemplos.assign(grupo = labels)
pd.options.display.max_colwidth = 5000
tweets_ag

tweets_ag.to_csv('knn.csv')

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(vec_text, y)

labels = clf.predict(vec_text)
tweets_svm = twitter_exemplos.assign(grupo = labels)
tweets_svm


tweets_svm.to_csv('svm.csv')


In [ ]:
#ERRO
#from sklearn import linear_model
#reg = linear_model.Lasso(alpha=0.1)
#reg.fit(X_train, y_train)

#labels = reg.predict(X_train)
#tweets_reg = twitter_exemplos.assign(grupo = labels)
#tweets_reg


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(vec_text, y)

labels = decision_tree.predict(vec_text)
tweets_tree = twitter_exemplos.assign(grupo = labels)
tweets_tree

tweets_tree.to_csv('tree.csv')


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb = gnb.fit(vec_text.toarray(), y)


labels = nb.predict(vec_text.toarray())
nb_tweets = twitter_exemplos.assign(grupo = labels)
nb_tweets

nb_tweets.to_csv('nb.csv')

# Recuperação

In [ ]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAGPcNAEAAAAAXgi8sXydvHWkhFM75Kam25WWrJE%3D5KuIZm1QrnRvIdCPyAKTT6XKnPjIFuQ2IOHc8Cz5GqBcIjfV4l"

In [ ]:
import requests

hed = {'Authorization': 'Bearer ' + bearer_token}

resp = requests.get("https://api.twitter.com/2/tweets/search/recent?query=python&tweet.fields=created_at,geo,public_metrics&expansions=geo.place_id&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type&user.fields=location", headers = hed)

resp.json()


{'data': [{'created_at': '2021-06-28T13:30:44.000Z',
   'id': '1409504540907511816',
   'public_metrics': {'like_count': 0,
    'quote_count': 0,
    'reply_count': 0,
    'retweet_count': 44},
   'text': 'RT @WajdiAlkayal: Using# Salesforce? #AI \n#100DaysOfCode #AI #Analytics #ArtificialIntelligence #BigData #Coding #DataScience #DeepLearning…'},
  {'created_at': '2021-06-28T13:30:43.000Z',
   'id': '1409504537606594561',
   'public_metrics': {'like_count': 0,
    'quote_count': 0,
    'reply_count': 0,
    'retweet_count': 3},
   'text': 'RT @0lympicker: In Python, for x mod 2k we can write: x % 2**k.\nIt turns out that this is equivalent to x &amp; (2k – 1) where &amp; stands for bin…'},
  {'created_at': '2021-06-28T13:30:42.000Z',
   'id': '1409504533588422662',
   'public_metrics': {'like_count': 0,
    'quote_count': 0,
    'reply_count': 0,
    'retweet_count': 1},
   'text': 'RT @NiPaix: #followback penis sexe liberal #designthinking #100DaysOfCode python #indiedev #RadialFirs

In [ ]:
resp2 = requests.get("https://api.twitter.com/1.1/geo/search.json?query=curitiba,transito,Curitiba,acidente,congestinamento&granularity=neighborhood&trim_place=false&tweet.fields=created_at,geo,public_metrics", headers=hed)
resp2.json()

{'query': {'params': {'granularity': 'neighborhood',
   'query': 'curitiba,transito,Curitiba,acidente,congestinamento',
   'trim_place': False},
  'type': 'search',
  'url': 'https://api.twitter.com/1.1/geo/search.json?query=curitiba,transito,Curitiba,acidente,congestinamento&granularity=neighborhood&trim_place=false&tweet.fields=created_at,geo,public_metrics'},
 'result': {'places': [{'attributes': {},
    'bounding_box': {'coordinates': [[[-49.3916434, -25.6447517],
       [-49.3916434, -25.3457471],
       [-49.1852775, -25.3457471],
       [-49.1852775, -25.6447517],
       [-49.3916434, -25.6447517]]],
     'type': 'Polygon'},
    'centroid': [-49.28463542494987, -25.4952494],
    'contained_within': [{'attributes': {},
      'bounding_box': {'coordinates': [[[-54.619501, -26.712827],
         [-54.619501, -22.516383],
         [-48.023474, -22.516383],
         [-48.023474, -26.712827],
         [-54.619501, -26.712827]]],
       'type': 'Polygon'},
      'centroid': [-51.8166505

In [ ]:
resp3 = requests.get("https://api.twitter.com/1.1/search/tweets.json?geocode=-25.456766,-49.272093,10km&q=Curitiba,transito", headers = hed)
resp3.json()

{'search_metadata': {'completed_in': 0.07,
  'count': 15,
  'max_id': 1409491633683447810,
  'max_id_str': '1409491633683447810',
  'next_results': '?max_id=1407711259189972994&q=Curitiba%2Ctransito&geocode=-25.456766%2C-49.272093%2C10km&include_entities=1',
  'query': 'Curitiba%2Ctransito',
  'refresh_url': '?since_id=1409491633683447810&q=Curitiba%2Ctransito&geocode=-25.456766%2C-49.272093%2C10km&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'},
 'statuses': [{'contributors': None,
   'coordinates': None,
   'created_at': 'Mon Jun 28 12:39:27 +0000 2021',
   'entities': {'hashtags': [],
    'symbols': [],
    'urls': [],
    'user_mentions': [{'id': 171502474,
      'id_str': '171502474',
      'indices': [3, 18],
      'name': 'TrânsitoSetran',
      'screen_name': 'TransitoSetran'}]},
   'favorite_count': 0,
   'favorited': False,
   'geo': None,
   'id': 1409491633683447810,
   'id_str': '1409491633683447810',
   'in_reply_to_screen_name': None,
   'in_reply_to_status_